# Neural Particles 2D

## 2D Dataset

### Setup

In [9]:
import os
from subprocess import Popen, PIPE

dir_loc = "2D_SPH/"

data_version = 0

def run_manta(scene, param={}):
    command = [dir_loc+"build/manta", dir_loc+scene]

    for k, v in param.items():
        command += [k, str(v)]
        
    print(" ".join(command) + "\n")

    proc = Popen(command, stdin=None, stdout=PIPE, stderr=PIPE)

    for line in proc.stdout:
        print(line.decode('utf-8'))
    for line in proc.stderr:
        print(line.decode('utf-8'))
    

### Generate High-res Data

In [30]:
highres_prefix = "2D_data/drop_2D_v%02d" % data_version

frame_count = 50

param = {}

#disable gui
param['gui'] = 1

# output file format
param['out'] = highres_prefix+"_%03d.uni"

# fixed dt
#param['dt'] = 0.001

# write only every 30th frame -> 30 frames are one timestep
param['fps'] = 30

# simulation time (how many frames)
param['t_end'] = float(frame_count) / param['fps']

run_manta("scenes/2D_drop.py", param)

2D_SPH/build/manta 2D_SPH/scenes/2D_drop.py out 2D_data/drop_2D_v00_%03d.uni gui 1 fps 30 t_end 1.6666666666666667

Version: mantaflow mac 64bit fp1 commit 1afafd3176b0065e417231e9366b148c12e91dcb from Oct 18 2017, 14:31:36

Loading script '2D_SPH/scenes/2D_drop.py'

Added & initialized 10176 particles

Added & initialized 368 particles

Added & initialized 8900 particles

   Frame 0, dt: 0.001926285, 0/0.03333334 lock:0

   Frame 0, dt: 0.001926285, 0.001926285/0.03333334 lock:0

writing particles pp to uni file 2D_data/drop_2D_v00_000.uni

   Frame 0, dt: 0.001926285, 0.00385257/0.03333334 lock:0

   Frame 0, dt: 0.001926285, 0.005778856/0.03333334 lock:0

   Frame 0, dt: 0.001926285, 0.007705141/0.03333334 lock:0

   Frame 0, dt: 0.001926285, 0.009631426/0.03333334 lock:0

   Frame 0, dt: 0.001926285, 0.01155771/0.03333334 lock:0

   Frame 0, dt: 0.001926285, 0.013484/0.03333334 lock:0

   Frame 0, dt: 0.001926285, 0.01541028/0.03333334 lock:0

   Frame 0, dt: 0.001926285, 0.0173365

### Generate Low-res Data

In [35]:
lowres_prefix = "2D_data/tools/l_drop_2D_v%02d" % data_version

param = {}
param['gui'] = 1
param['in'] = highres_prefix+"_%03d.uni"
param['out'] = lowres_prefix+"_%03d.uni"
param['pause'] = 1

run_manta("scenes/tools/downsample.py", param)

2D_SPH/build/manta 2D_SPH/scenes/tools/downsample.py out 2D_data/tools/l_drop_2D_v00_%03d.uni in 2D_data/drop_2D_v00_%03d.uni pause 1 gui 1

Version: mantaflow mac 64bit fp1 commit 1afafd3176b0065e417231e9366b148c12e91dcb from Oct 18 2017, 14:31:36

Loading script '2D_SPH/scenes/tools/downsample.py'

Step: Gui::pause

reading particles pp from uni file 2D_data/drop_2D_v00_000.uni

 Deleted 17541 particles

reading particles pp from uni file 2D_data/drop_2D_v00_001.uni

 Deleted 17541 particles

reading particles pp from uni file 2D_data/drop_2D_v00_002.uni

 Deleted 17541 particles

reading particles pp from uni file 2D_data/drop_2D_v00_003.uni

 Deleted 17541 particles

reading particles pp from uni file 2D_data/drop_2D_v00_004.uni

 Deleted 17541 particles

reading particles pp from uni file 2D_data/drop_2D_v00_005.uni

 Deleted 17541 particles

reading particles pp from uni file 2D_data/drop_2D_v00_006.uni

 Deleted 17541 particles

reading particles pp from uni file 2D_data/drop_2D

## Train Neural Network

### Setup Neural Network

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

conv_f = 3
conv_k = 3
conv_s = (1, 1)

deconv_f = 3
deconv_k = 3
deconv_s = (1, 1)

input_shape = (64, 64, 1)

model = Sequential()

model.add(keras.layers.Conv2D(input_shape=input_shape, filters=conv_f,            \
                              kernel_size=conv_k, strides=conv_s,                 \
                              padding='valid', data_format=None,                  \
                              dilation_rate=(1, 1), activation=None,              \
                              use_bias=True, kernel_initializer='glorot_uniform', \
                              bias_initializer='zeros', kernel_regularizer=None,  \
                              bias_regularizer=None, activity_regularizer=None,   \
                              kernel_constraint=None, bias_constraint=None))

model.add(Dense(units=100))
model.add(Activation('relu'))

model.add(keras.layers.Conv2DTranspose(filters=deconv_f, kernel_size=deconv_k,    \
                                       strides=deconv_s, padding='valid',         \
                                       data_format=None, activation=None,         \
                                       use_bias=True,                             \
                                       kernel_initializer='glorot_uniform',       \
                                       bias_initializer='zeros',                  \
                                       kernel_regularizer=None,                   \
                                       bias_regularizer=None,                     \
                                       activity_regularizer=None,                 \
                                       kernel_constraint=None,                    \
                                       bias_constraint=None))

Using TensorFlow backend.
